## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import sagemaker, boto3
import sagemaker.amazon.common as smac
sage_client = boto3.Session().client('sagemaker')
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

##  HyperParameter Tuning Job Analytics

In [2]:
tuning_job_name='knn-HPO-210110-0945'

# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

5 training jobs have completed


## HPO Tuner and its best model

In [3]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2021, 1, 10, 9, 52, 44, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'test:mse',
                                                 'Value': 3222687.0},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 1, 10, 9, 56, 4, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:719009365707:training-job/knn-hpo-210110-0945-005-f5e30bdb',
 'TrainingJobName': 'knn-HPO-210110-0945-005-f5e30bdb',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 1, 10, 9, 54, 48, tzinfo=tzlocal()),
 'TunedHyperParameters': {'k': '10', 'sample_size': '9222'}}


## Tabulating HPO training job analytics

In [4]:
tuning_job_name='knn-HPO-210110-0945'
tunerresult = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
full_df = tunerresult.dataframe()

full_df = tunerresult.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df


Number of training jobs with valid objective: 5
{'highest': 3774818.5, 'lowest': 3222687.0}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,k,sample_size
0,3222687.0,76.0,2021-01-10 09:56:04-05:00,knn-HPO-210110-0945-005-f5e30bdb,Completed,2021-01-10 09:54:48-05:00,10.0,9222.0
1,3274660.0,74.0,2021-01-10 09:52:27-05:00,knn-HPO-210110-0945-004-08ff9ce4,Completed,2021-01-10 09:51:13-05:00,9.0,17227.0
2,3274660.0,70.0,2021-01-10 09:52:31-05:00,knn-HPO-210110-0945-003-079e2afe,Completed,2021-01-10 09:51:21-05:00,9.0,14434.0
4,3293788.5,72.0,2021-01-10 09:48:47-05:00,knn-HPO-210110-0945-001-24a3417d,Completed,2021-01-10 09:47:35-05:00,8.0,13114.0
3,3774818.5,73.0,2021-01-10 09:48:41-05:00,knn-HPO-210110-0945-002-77fc89c3,Completed,2021-01-10 09:47:28-05:00,3.0,10950.0


## Plotting the graph with results from Hyperparameter tuner

In [5]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tunerresult)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)


Loading BokehJS ...

## Plotting the graph with results from Hyperparameter tuner

In [6]:
ranges = tunerresult.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))